# Importando as Bibliotecas e o DataFrame 

In [ ]:
import pandas as pd
from google.colab import data_table
data_table.enable_dataframe_formatter()
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# DataFrame 2021/2022
df = pd.read_excel("https://github.com/futpythontrader/YouTube/blob/main/DataFrame_2021_2022.xlsx?raw=true")

# DataFrame 2020/2021
# df = pd.read_excel("https://github.com/futpythontrader/YouTube/blob/main/DataFrame_2020_2021.xlsx?raw=true")

In [ ]:
# Pontos
def Pontos_H (Result_FT):
    if Result_FT == 'H':
        Pontos = 3
    elif Result_FT == 'D':
        Pontos = 1
    else:
        Pontos = 0
    return Pontos

def Pontos_A (Result_FT):
    if Result_FT == 'A':
        Pontos = 3
    elif Result_FT == 'D':
        Pontos = 1
    else:
        Pontos = 0
    return Pontos

df['Pontos_H'] = df.apply(lambda row: Pontos_H(row['Result_FT']), axis=1)
df['Pontos_A'] = df.apply(lambda row: Pontos_A(row['Result_FT']), axis=1)

In [ ]:
# Médias de Pontos
df['Media_Pontos_H'] = df.groupby('Home')['Pontos_H'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True).shift(1)
df['Media_Pontos_A'] = df.groupby('Away')['Pontos_A'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True).shift(1)

# Desvio padrão
df['DP_Pontos_H'] = df.groupby('Home')['Pontos_H'].rolling(window=5, min_periods=1).std().reset_index(0,drop=True).shift(1)
df['DP_Pontos_A'] = df.groupby('Away')['Pontos_A'].rolling(window=5, min_periods=1).std().reset_index(0,drop=True).shift(1)

# Coeficiente de variação
df['CV_Pontos_H'] = df['DP_Pontos_H'] / df['Media_Pontos_H']
df['CV_Pontos_A'] = df['DP_Pontos_A'] / df['Media_Pontos_A']

In [ ]:
# Probabilidades
df['p(H)'] = df.apply(lambda row: (1/ row['Odds_H']), axis=1)
df['p(D)'] = df.apply(lambda row: (1/ row['Odds_D']), axis=1)
df['p(A)'] = df.apply(lambda row: (1/ row['Odds_A']), axis=1)

# Valor do Gol
df['Valor_do_Gol_H'] = df.apply(lambda row: (row['Goals_H_FT'] * row['p(A)']), axis=1)
df['Valor_do_Gol_A'] = df.apply(lambda row: (row['Goals_A_FT'] * row['p(H)']), axis=1)

# Custo do Gol
df['Custo_do_Gol_H'] = df.apply(lambda row: (row['p(H)'] / row['Goals_H_FT']), axis=1)
df['Custo_do_Gol_A'] = df.apply(lambda row: (row['p(A)'] / row['Goals_A_FT']), axis=1)

def custo ():
Try:
df.p(H)/df.Goals_H_FT


# Média do Valor do Gol
df['Media_Valor_Gol_H'] = df.groupby('Home')['Valor_do_Gol_H'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True).shift(1)
df['Media_Valor_Gol_A'] = df.groupby('Away')['Valor_do_Gol_A'].rolling(window=5, min_periods=1).mean().reset_index(0,drop=True).shift(1)

In [ ]:
df['Custo_do_Gol_H']

# Ajustando o DataFrame

In [ ]:
df1 = df[['Home','Away','Odds_H','Odds_D','Odds_A','Odds_Over25_FT','Goals_H_FT','Goals_A_FT','Result_FT','Media_Pontos_H','Media_Pontos_A','CV_Pontos_H','CV_Pontos_A']]
df1 = df1.dropna()

In [ ]:
df1

In [ ]:
df1.info()

# Filtro Estatístico

In [ ]:
# Filtros
flt = (df1.Odds_H > 1.10) & (df1.Odds_H < 2.20) & (df1.Media_Pontos_H > 2.5) & (df1.CV_Pontos_H < 0.7) 
df2 = df1[flt];

# Ajustando o Índice
df2.reset_index(inplace=True, drop=True)
df2.index = df2.index.set_names(['Nº'])
df2 = df2.rename(index=lambda x: x + 1)

# Resultados

In [ ]:
Stake = 1
green = Stake * (df2.Odds_H - 1) 
red = -Stake

# Back Home
df2.loc[((df2['Result_FT']) == "H"), 'Profit_Back_Home'] = green
df2.loc[((df2['Result_FT']) != "H"), 'Profit_Back_Home'] = red
df2['Profit_acu_Back_Home'] = df2.Profit_Back_Home.cumsum()
df2.Profit_acu_Back_Home.plot(title='Back Home', xlabel='Quantidade de Apostas', ylabel='Stakes')

In [ ]:
df2['Profit_acu_Back_Home'].tail(1)

In [ ]:
df2.head(10)

In [ ]:
# Filtros
flt = (df1.Odds_H > 1.10) & (df1.Odds_H < 1.50) & (df1.Media_Pontos_H > 1) & (df1.Media_Pontos_A < 1) & (df1.CV_Pontos_H < 0.75) & (df1.Odds_Over25_FT < 2)
df2 = df1[flt];

# Ajustando o Índice
df2.reset_index(inplace=True, drop=True)
df2.index = df2.index.set_names(['Nº'])
df2 = df2.rename(index=lambda x: x + 1)

Stake = 1
green = Stake * (df2.Odds_H - 1) 
red = -Stake

# Back Home
df2.loc[((df2['Result_FT']) == "H"), 'Profit_Back_Home'] = green
df2.loc[((df2['Result_FT']) != "H"), 'Profit_Back_Home'] = red
df2['Profit_acu_Back_Home'] = df2.Profit_Back_Home.cumsum()
df2.Profit_acu_Back_Home.plot(title='Back Home', xlabel='Quantidade de Apostas', ylabel='Stakes')

df2['Profit_acu_Back_Home'].tail(1)

In [ ]:
df2